In [2]:
import numpy as np 
import cv2

STAGE_FIRST_FRAME = 0
STAGE_SECOND_FRAME = 1
STAGE_DEFAULT_FRAME = 2
kMinNumFeature = 1500

lk_params = dict(winSize  = (21, 21), 
                #maxLevel = 3,
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))

def featureTracking(image_ref, image_cur, px_ref):
    kp2, st, err = cv2.calcOpticalFlowPyrLK(image_ref, image_cur, px_ref, None, **lk_params)  #shape: [k,2] [k,1] [k,1]

    st = st.reshape(st.shape[0])
    kp1 = px_ref[st == 1]
    kp2 = kp2[st == 1]

    return kp1, kp2


class PinholeCamera:
    def __init__(self, width, height, fx, fy, cx, cy, 
                k1=0.0, k2=0.0, p1=0.0, p2=0.0, k3=0.0):
        self.width = width
        self.height = height
        self.fx = fx
        self.fy = fy
        self.cx = cx
        self.cy = cy
        self.distortion = (abs(k1) > 0.0000001)
        self.d = [k1, k2, p1, p2, k3]


class VisualOdometry:
    def __init__(self, cam):
        self.frame_stage = 0
        self.cam = cam
        self.new_frame = None
        self.last_frame = None
        self.cur_R = None
        self.cur_t = None
        self.px_ref = None
        self.px_cur = None
        self.focal = cam.fx
        self.pp = (cam.cx, cam.cy)
        self.trueX, self.trueY, self.trueZ = 0, 0, 0
        self.detector = cv2.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)

    def processFirstFrame(self):
        self.px_ref = self.detector.detect(self.new_frame)
        self.px_ref = np.array([x.pt for x in self.px_ref], dtype=np.float32)
        self.frame_stage = STAGE_SECOND_FRAME

    def processSecondFrame(self):
        self.px_ref, self.px_cur = featureTracking(self.last_frame, self.new_frame, self.px_ref)
        E, mask = cv2.findEssentialMat(self.px_cur, self.px_ref, focal=self.focal, pp=self.pp, method=cv2.RANSAC, prob=0.999, threshold=1.0)
        _, self.cur_R, self.cur_t, mask = cv2.recoverPose(E, self.px_cur, self.px_ref, focal=self.focal, pp = self.pp)
        self.frame_stage = STAGE_DEFAULT_FRAME 
        self.px_ref = self.px_cur

    def processFrame(self, frame_id):
        self.px_ref, self.px_cur = featureTracking(self.last_frame, self.new_frame, self.px_ref)
        E, mask = cv2.findEssentialMat(self.px_cur, self.px_ref, focal=self.focal, pp=self.pp, method=cv2.RANSAC, prob=0.999, threshold=1.0)
        _, R, t, mask = cv2.recoverPose(E, self.px_cur, self.px_ref, focal=self.focal, pp = self.pp)
        self.cur_t = self.cur_t + 4*self.cur_R.dot(t) 
        self.cur_R = R.dot(self.cur_R)
        if(self.px_ref.shape[0] < kMinNumFeature):
            self.px_cur = self.detector.detect(self.new_frame)
            self.px_cur = np.array([x.pt for x in self.px_cur], dtype=np.float32)
        self.px_ref = self.px_cur

    def update(self, img, frame_id):
        assert(img.ndim==2 and img.shape[0]==self.cam.height and img.shape[1]==self.cam.width), "Frame: provided image has not the same size as the camera model or image is not grayscale"
        self.new_frame = img
        if(self.frame_stage == STAGE_DEFAULT_FRAME):
            self.processFrame(frame_id)
        elif(self.frame_stage == STAGE_SECOND_FRAME):
            self.processSecondFrame()
        elif(self.frame_stage == STAGE_FIRST_FRAME):
            self.processFirstFrame()
        self.last_frame = self.new_frame

In [ ]:
from __future__ import print_function
import numpy as np
import cv2
import pyzbar.pyzbar as pyzbar
import matplotlib.pyplot as plt
from matplotlib.image import imread, imsave
from PIL import Image 
import os

#cap = cv2.VideoCapture('QR_short.mp4')
cap = cv2.VideoCapture('v3.mp4')

#fourcc = cv2.VideoWriter_fourcc(*'MJPG') # ('M','J','P','G') # 'F','M','P','4'
fourcc = cv2.VideoWriter_fourcc('F','M','P','4')

#specifying the video output resolution to exactly the same as input
out = cv2.VideoWriter('QR_detected.avi',fourcc, 20.0,(int(cap.get(3)),int(cap.get(4))))
#out = cv2.VideoWriter('QR_detected.avi', fourcc, 24.0, (640,480))

# for logitech
#cam = PinholeCamera(1920, 1088, 865, 875, 984, 411)

# for Samsung phone
cam = PinholeCamera(1920, 1080, 1236, 1208, 917, 520)

vo = VisualOdometry(cam)


def decode(img) : 
  # Find barcodes and QR codes

    decodedObjects = pyzbar.decode(img)
    return decodedObjects


def display(img, decodedObjects):
    # Display barcode and QR code location
    # collect them into dict
    
    QRs_dict = {}

    for i, decodedObject in enumerate(decodedObjects): 
        points = decodedObject.polygon
        if len(points) == 4 :
            center_x = 0
            center_y = 0
            for point in points:
                center_x += point[0]
                center_y += point[1]
            QRs_dict.update({str(decodedObject.data) : [center_x/len(points), center_y/len(points)]})
            
            
            
# for convex hull, not in use by now
        
#         # If the points do not form a quad, find convex hull
#         if len(points) > 4 : 
#             print("one hull detected")
#             hull = cv2.convexHull(np.array([point for point in points], dtype=np.float32))
#             hull = list(map(tuple, np.squeeze(hull)))
#             print("hull is ", hull)
#         else : 
#             hull = points;

#         # Number of points in the convex hull
#         n = len(hull)

#         # Draw the convext hull
#         for j in range(0,n):
#             cv2.line(img, hull[j], hull[ (j+1) % n], (255,0,0), 3)


        for value in QRs_dict.values():
            #cv2.circle(img,(int(value[0]), int(value[1])), 15, (0,0,0), -1)
            cv2.circle(img,(int(value[0]), int(value[1])), 15, (255,255,255), -1)
            
    return QRs_dict


def auto_bw(img):
    
    # assert img.shape 
    
    img = img
    imgout = np.sort(img.flatten().astype('float'))
    k = round(imgout.size*0.05)
    coeff = 255/(imgout[-k] - imgout[k])
    img = np.clip(img - imgout[k], 0, 255)
    imgout = np.clip(img*coeff, 0, 255)
    #imsave("out_img.png", imgout.astype('uint8'))
    return imgout.astype('uint8')



def find_qr_codes(img):
    
    decodedObjects = decode(img)
    QRs_dict = display(img, decodedObjects)
    
    return img, QRs_dict



# def read_img(path):
#     """Given a path to an image file, returns a cv2 array
#     str -> np.ndarray"""
#     if os.path.isfile(path):
#         return cv2.imread(path)
#     else:
#         raise ValueError('Path provided is not a valid file: {}'.format(path))

img_id = 0

landmarks_all = []

Rs = []
ts = []

while(cap.isOpened()):
    
    _ , frame = cap.read()
    
    if _ is True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        vo.update(gray, img_id)
        Rs.append(vo.cur_R)
        ts.append(vo.cur_t)
        
        # equalizes histogramm of an image
        bw = auto_bw(gray) 
        
        frame_with_detected_QRs, landmarks_on_frame = find_qr_codes(bw)
        print("img_id is", img_id)
        print(landmarks_on_frame)
        print(Rs[-1], ts[-1])
        print('----------------------------------------------')
        landmarks_all.append(landmarks_on_frame)
        out.write(frame_with_detected_QRs)
        cv2.imshow('frame', frame_with_detected_QRs)
        img_id += 1
        imsave("processed_img_{}.png".format(img_id), frame_with_detected_QRs.astype('uint8'))
        
    else:
        continue
    
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

img_id is 0
{"b'6'": [1252.5, 63.25]}
None None
----------------------------------------------
img_id is 1
{"b'4'": [1087.75, 615.75], "b'2'": [681.0, 979.25], "b'6'": [1254.0, 63.25]}
[[-0.94052116 -0.22302055 -0.25628457]
 [-0.22115726 -0.17072733  0.96017792]
 [-0.25789419  0.95974685  0.11125005]] [[-0.17255209]
 [ 0.64335658]
 [ 0.74586733]]
----------------------------------------------
img_id is 2
{"b'2'": [683.75, 980.0], "b'4'": [1090.25, 616.25], "b'6'": [1256.75, 62.25]}
[[-0.94014869 -0.22438644 -0.25645891]
 [-0.22125203 -0.17043795  0.9602075 ]
 [-0.25916787  0.95947988  0.11059106]] [[-0.65124462]
 [-3.1809115 ]
 [-0.32456577]]
----------------------------------------------
img_id is 3
{"b'2'": [685.25, 981.75], "b'4'": [1092.25, 617.5], "b'6'": [1260.0, 63.5]}
[[-0.93993496 -0.22601853 -0.25580833]
 [-0.22021552 -0.17109952  0.96032811]
 [-0.26082063  0.95897893  0.11104964]] [[-0.96389831]
 [ 0.76899164]
 [ 0.2236247 ]]
----------------------------------------------
im

img_id is 30
{"b'2'": [737.0, 1009.5]}
[[ 0.8773822   0.46296942  0.12593567]
 [-0.47693378  0.87018369  0.12375187]
 [-0.05229383 -0.16864066  0.98428943]] [[-0.567252  ]
 [-1.94132287]
 [21.82251577]]
----------------------------------------------
img_id is 31
{"b'2'": [739.25, 1009.5], "b'6'": [1332.0, 76.25]}
[[-0.26885946  0.24143073 -0.93243005]
 [ 0.65580008 -0.66313683 -0.36079882]
 [-0.70543663 -0.70849188  0.01996032]] [[-2.67841596]
 [-1.01451191]
 [25.09115588]]
----------------------------------------------
img_id is 32
{"b'2'": [742.25, 1010.25], "b'6'": [1336.5, 74.0]}
[[-0.26737282  0.24330433 -0.93237052]
 [ 0.6561118  -0.66267637 -0.36107801]
 [-0.70571175 -0.70828175  0.01754681]] [[-5.18677129]
 [-3.49753949]
 [23.2089424 ]]
----------------------------------------------
img_id is 33
{"b'2'": [745.75, 1011.0], "b'4'": [1158.75, 641.0], "b'6'": [1341.0, 75.5]}
[[ 0.77346098 -0.13507704 -0.6192837 ]
 [-0.34515091  0.72971433 -0.59024388]
 [ 0.53162859  0.67027695  0.5

img_id is 61
{"b'4'": [1275.75, 680.5], "b'6'": [1512.0, 90.5]}
[[-0.98283209 -0.10762849  0.14985723]
 [ 0.12235743  0.22770165  0.96601274]
 [-0.13809323  0.96776447 -0.21062334]] [[-10.58092961]
 [ -2.29988062]
 [ 29.01568795]]
----------------------------------------------
img_id is 62
{}
[[-0.98346461 -0.10108639  0.15026276]
 [ 0.1244714   0.22538659  0.96628555]
 [-0.13154553  0.96901106 -0.20907737]] [[-6.73198112]
 [-2.3272837 ]
 [30.10419361]]
----------------------------------------------
img_id is 63
{}
[[-0.98345971 -0.10073145  0.15053296]
 [ 0.12479726  0.22549628  0.96621792]
 [-0.13127315  0.9690225  -0.2091955 ]] [[-10.55422575]
 [ -3.03241418]
 [ 31.04930578]]
----------------------------------------------
img_id is 64
{}
[[-0.98328438 -0.1016787   0.15104059]
 [ 0.12499657  0.22621819  0.96602339]
 [-0.13239214  0.96875527 -0.20972732]] [[-14.07747469]
 [ -4.77268554]
 [ 31.79641517]]
----------------------------------------------
img_id is 65
{}
[[-0.9828048  -0.10

In [ ]:
img_id is 0
{"b'6'": [1252.5, 63.25]}
None None
----------------------------------------------
img_id is 1
{"b'4'": [1087.75, 615.75], "b'2'": [681.0, 979.25], "b'6'": [1254.0, 63.25]}
[[-0.94052116 -0.22302055 -0.25628457]
 [-0.22115726 -0.17072733  0.96017792]
 [-0.25789419  0.95974685  0.11125005]] [[-0.17255209]
 [ 0.64335658]
 [ 0.74586733]]
----------------------------------------------
img_id is 2
{"b'2'": [683.75, 980.0], "b'4'": [1090.25, 616.25], "b'6'": [1256.75, 62.25]}
[[-0.94014869 -0.22438644 -0.25645891]
 [-0.22125203 -0.17043795  0.9602075 ]
 [-0.25916787  0.95947988  0.11059106]] [[-0.65124462]
 [-3.1809115 ]
 [-0.32456577]]
----------------------------------------------
img_id is 3
{"b'2'": [685.25, 981.75], "b'4'": [1092.25, 617.5], "b'6'": [1260.0, 63.5]}
[[-0.93993496 -0.22601853 -0.25580833]
 [-0.22021552 -0.17109952  0.96032811]
 [-0.26082063  0.95897893  0.11104964]] [[-0.96389831]
 [ 0.76899164]
 [ 0.2236247 ]]
----------------------------------------------
img_id is 4
{"b'4'": [1092.5, 618.5], "b'2'": [684.5, 982.5], "b'6'": [1261.0, 64.5]}
[[-0.93987158 -0.22641651 -0.25568924]
 [-0.21957586 -0.17282672  0.96016528]
 [-0.2615872   0.95857524  0.11271932]] [[ 1.39826736]
 [-0.93821815]
 [-2.51601178]]
----------------------------------------------
img_id is 5
{"b'4'": [1093.0, 618.75], "b'2'": [684.75, 983.5], "b'6'": [1261.5, 64.25]}
[[-0.93964739 -0.22676628 -0.25620273]
 [-0.22002816 -0.17293766  0.96004176]
 [-0.2620122   0.95847255  0.11260545]] [[ 0.01547239]
 [ 2.81222111]
 [-2.66461667]]
----------------------------------------------
img_id is 6
{"b'4'": [1094.25, 619.5], "b'2'": [686.25, 983.75], "b'6'": [1263.5, 64.25]}
[[ 0.79642161  0.30218978  0.52382626]
 [-0.58691328  0.59501531  0.54908067]
 [-0.14575808 -0.7447403   0.65124225]] [[ 1.85883321]
 [ 1.84738379]
 [-6.08091728]]
----------------------------------------------
img_id is 7
{"b'4'": [1096.0, 619.5], "b'2'": [687.75, 982.5], "b'6'": [1265.5, 64.75]}
[[ 0.79647703  0.30372565  0.52285281]
 [-0.5872      0.59485617  0.54894653]
 [-0.14429308 -0.74424248  0.65213698]] [[ 3.04504948]
 [ 4.14284519]
 [-3.027436  ]]
----------------------------------------------
img_id is 8
{"b'4'": [1097.0, 619.5], "b'6'": [1266.75, 64.5]}
[[-0.81014146 -0.38689247 -0.44043733]
 [ 0.57835071 -0.65027091 -0.4926035 ]
 [-0.095819   -0.65380575  0.75057095]] [[ 5.36566897]
 [ 6.29026987]
 [-0.57727428]]
----------------------------------------------
img_id is 9
{"b'2'": [692.5, 983.25], "b'4'": [1100.0, 619.5], "b'6'": [1269.75, 64.5]}
[[-0.81013734 -0.38716033 -0.44020946]
 [ 0.57843893 -0.6499986  -0.49285923]
 [-0.09531999 -0.65391795  0.75053675]] [[ 2.21240566]
 [ 8.70817387]
 [-1.03626316]]
----------------------------------------------
img_id is 10
{"b'6'": [1272.75, 65.75]}
[[-0.14798277  0.93323836  0.3273641 ]
 [ 0.75016603 -0.10978279  0.65207259]
 [ 0.6444781   0.34207294 -0.68383776]] [[ 2.16473781]
 [12.61723423]
 [-0.18951947]]
----------------------------------------------
img_id is 11
{}
[[-0.15198944  0.93227782  0.32826404]
 [ 0.75055159 -0.10723175  0.65205342]
 [ 0.64309526  0.34548433 -0.68342451]] [[-1.31003447]
 [10.81753087]
 [-1.0183894 ]]
----------------------------------------------
img_id is 12
{"b'6'": [1282.5, 67.0]}
[[-0.15515989  0.93102308  0.33033532]
 [ 0.7496756  -0.10679941  0.65313121]
 [ 0.64335985  0.3489841  -0.68139431]] [[ 0.54597243]
 [13.57500526]
 [-3.24360249]]
----------------------------------------------
img_id is 13
{"b'6'": [1286.75, 69.0]}
[[-0.15596577  0.93041304  0.33167192]
 [ 0.74876863 -0.10763233  0.65403426]
 [ 0.64422062  0.35035248 -0.67987714]] [[ 0.53234019]
 [14.4952357 ]
 [ 0.64908168]]
----------------------------------------------
img_id is 14
{"b'2'": [711.75, 992.0], "b'4'": [1119.75, 627.5], "b'6'": [1290.0, 71.25]}
[[-0.15646335  0.93014634  0.33218519]
 [ 0.74805401 -0.10801321  0.65478877]
 [ 0.64492977  0.35094291 -0.6788996 ]] [[-0.39729355]
 [11.59801469]
 [-1.94743358]]
----------------------------------------------
img_id is 15
{"b'4'": [1119.25, 628.25], "b'2'": [711.0, 992.5], "b'6'": [1290.25, 72.25]}
[[-0.6157798   0.60561574  0.50402859]
 [-0.66512376 -0.05659963 -0.74458503]
 [-0.42240458 -0.79374181  0.43766233]] [[-1.74572268]
 [13.77837327]
 [ 1.12303467]]
----------------------------------------------
img_id is 16
{"b'2'": [710.0, 993.25], "b'4'": [1118.75, 628.5], "b'6'": [1290.25, 72.25]}
[[-0.16719083 -0.13098081 -0.97718537]
 [-0.58290309  0.8124899  -0.00917383]
 [ 0.79515484  0.56807059 -0.21218998]] [[-1.96823615]
 [14.60283116]
 [ 5.03081626]]
----------------------------------------------
img_id is 17
{"b'4'": [1117.75, 630.5], "b'6'": [1290.0, 74.0]}
[[-0.1667218  -0.13022871 -0.97736601]
 [-0.58111078  0.81377129 -0.00930311]
 [ 0.79656393  0.5664069  -0.21135072]] [[-2.35528323]
 [11.25630955]
 [ 2.8742014 ]]
----------------------------------------------
img_id is 18
{"b'4'": [1117.25, 630.75], "b'2'": [708.5, 995.75], "b'6'": [1289.25, 75.0]}
[[-0.16450131 -0.12867059 -0.97794846]
 [-0.58131731  0.81362419 -0.00926639]
 [ 0.79687484  0.56697403 -0.20864069]] [[-2.9209603 ]
 [ 8.98265352]
 [ 6.11619118]]
----------------------------------------------
img_id is 19
{"b'2'": [711.25, 995.25]}
[[-0.16510593 -0.12918068 -0.97777931]
 [-0.58060622  0.81412886 -0.00951972]
 [ 0.79726812  0.56613298 -0.20942061]] [[-2.18948998]
 [ 7.97451495]
 [ 9.91732362]]
----------------------------------------------
img_id is 20
{"b'6'": [1297.0, 72.25]}
[[-0.27906543 -0.13306878  0.95100746]
 [ 0.43389009 -0.90096494  0.0012549 ]
 [ 0.85665739  0.41298291  0.30916539]] [[-5.73587424]
 [ 8.2630662 ]
 [ 8.08978515]]
----------------------------------------------
img_id is 21
{"b'2'": [720.5, 998.0]}
[[ 0.57040277  0.35839973 -0.73904689]
 [-0.68206512  0.70800389 -0.18307829]
 [ 0.45763287  0.60850647  0.64829934]] [[-2.22079156]
 [ 9.28912697]
 [ 9.69956558]]
----------------------------------------------
img_id is 22
{"b'2'": [723.5, 999.25], "b'4'": [1132.25, 632.25], "b'6'": [1304.75, 73.0]}
[[ 0.56876092  0.35713622 -0.74092154]
 [-0.6826827   0.70741024 -0.18307124]
 [ 0.45875412  0.60993809  0.64615802]] [[-4.24036759]
 [ 7.38638362]
 [12.58069042]]
----------------------------------------------
img_id is 23
{"b'4'": [1135.25, 633.5], "b'2'": [726.25, 1001.0], "b'6'": [1308.5, 73.25]}
[[ 0.56932142  0.35853633 -0.73981405]
 [-0.6825159   0.70779354 -0.18220961]
 [ 0.45830684  0.60867069  0.64766877]] [[-0.86754255]
 [ 6.0766175 ]
 [14.28614166]]
----------------------------------------------
img_id is 24
{"b'2'": [728.0, 1002.25], "b'6'": [1312.25, 74.25]}
[[ 0.56836622  0.3580008  -0.74080717]
 [-0.68299885  0.70733251 -0.18219028]
 [ 0.45877273  0.60952125  0.64653804]] [[ 2.67124533]
 [ 4.50959539]
 [15.29679593]]
----------------------------------------------
img_id is 25
{"b'4'": [1139.75, 635.5], "b'2'": [729.0, 1003.0], "b'6'": [1315.0, 74.5]}
[[-0.47603631  0.29234435 -0.82941196]
 [ 0.72634926 -0.40100026 -0.55822535]
 [-0.49578844 -0.86817831 -0.02145342]] [[ 6.18635531]
 [ 3.56741622]
 [16.95700684]]
----------------------------------------------
img_id is 26
{"b'2'": [730.25, 1004.5], "b'4'": [1141.5, 636.25], "b'6'": [1317.0, 74.0]}
[[-0.47589203  0.29234389 -0.82949492]
 [ 0.72621439 -0.40140657 -0.55810879]
 [-0.49612441 -0.86799068 -0.02127815]] [[ 6.46667336]
 [ 6.06912782]
 [13.84849258]]
----------------------------------------------
img_id is 27
{"b'4'": [1142.5, 638.5], "b'2'": [731.0, 1007.5], "b'6'": [1319.5, 76.0]}
[[-0.47491823  0.29244545 -0.83001707]
 [ 0.72770334 -0.39987601 -0.55726746]
 [-0.49487425 -0.86866267 -0.02290512]] [[ 3.39560033]
 [ 3.55274793]
 [13.36234164]]
----------------------------------------------
img_id is 28
{"b'2'": [729.75, 1008.5], "b'4'": [1142.5, 639.25], "b'6'": [1320.0, 77.0]}
[[-0.47560093  0.29139326 -0.82999622]
 [ 0.72745459 -0.40021435 -0.55734934]
 [-0.49458424 -0.86886043 -0.02163305]] [[ 1.18660631]
 [ 1.08362955]
 [15.6037251 ]]
----------------------------------------------
img_id is 29
{"b'6'": [1322.0, 76.0]}
[[-0.47651108  0.29162709 -0.82939184]
 [ 0.72588712 -0.40173264 -0.5582999 ]
 [-0.49600914 -0.86808095 -0.0202583 ]] [[ 1.29194395]
 [ 0.81052216]
 [19.59300027]]
----------------------------------------------
img_id is 30
{"b'2'": [737.0, 1009.5]}
[[ 0.8773822   0.46296942  0.12593567]
 [-0.47693378  0.87018369  0.12375187]
 [-0.05229383 -0.16864066  0.98428943]] [[-0.567252  ]
 [-1.94132287]
 [21.82251577]]
----------------------------------------------
img_id is 31
{"b'2'": [739.25, 1009.5], "b'6'": [1332.0, 76.25]}
[[-0.26885946  0.24143073 -0.93243005]
 [ 0.65580008 -0.66313683 -0.36079882]
 [-0.70543663 -0.70849188  0.01996032]] [[-2.67841596]
 [-1.01451191]
 [25.09115588]]
----------------------------------------------
img_id is 32
{"b'2'": [742.25, 1010.25], "b'6'": [1336.5, 74.0]}
[[-0.26737282  0.24330433 -0.93237052]
 [ 0.6561118  -0.66267637 -0.36107801]
 [-0.70571175 -0.70828175  0.01754681]] [[-5.18677129]
 [-3.49753949]
 [23.2089424 ]]
----------------------------------------------
img_id is 33
{"b'2'": [745.75, 1011.0], "b'4'": [1158.75, 641.0], "b'6'": [1341.0, 75.5]}
[[ 0.77346098 -0.13507704 -0.6192837 ]
 [-0.34515091  0.72971433 -0.59024388]
 [ 0.53162859  0.67027695  0.5177836 ]] [[-4.52426649]
 [-2.2711166 ]
 [19.45967923]]
----------------------------------------------
img_id is 34
{"b'4'": [1162.0, 640.75], "b'2'": [749.25, 1010.25], "b'6'": [1345.75, 77.25]}
[[ 0.79228663 -0.14615564  0.59238537]
 [-0.55787802 -0.56670455  0.60631515]
 [ 0.24709111 -0.81085416 -0.53052946]] [[-2.79458331]
 [-5.64377998]
 [20.73770031]]
----------------------------------------------
img_id is 35
{"b'4'": [1164.75, 641.0], "b'2'": [751.0, 1010.25], "b'6'": [1349.75, 76.25]}
[[ 0.79111821 -0.1462614   0.59391883]
 [-0.55897962 -0.56711155  0.6049184 ]
 [ 0.24834202 -0.81055049 -0.53040942]] [[-1.85251595]
 [-2.38045484]
 [18.62506732]]
----------------------------------------------
img_id is 36
{"b'2'": [752.75, 1013.75], "b'4'": [1168.25, 643.0]}
[[-0.98538835  0.13946594  0.09777039]
 [ 0.15957975  0.55533799  0.81617034]
 [ 0.05953235  0.81984692 -0.56947953]] [[-1.19881968]
 [ 1.47989123]
 [19.44385266]]
----------------------------------------------
img_id is 37
{"b'4'": [1170.5, 644.25], "b'2'": [753.75, 1016.25], "b'6'": [1359.25, 77.0]}
[[ 0.67238798 -0.39748225  0.62442154]
 [-0.52199413 -0.85273121  0.01927713]
 [ 0.52480142 -0.33890609 -0.78084962]] [[ 0.46988923]
 [ 2.68034703]
 [16.01248028]]
----------------------------------------------
img_id is 38
{"b'4'": [1172.75, 644.75], "b'2'": [754.75, 1017.75]}
[[ 0.67028067 -0.39798802  0.626362  ]
 [-0.5226208  -0.85238222  0.01766504]
 [ 0.52686936 -0.33919034 -0.77933215]] [[-2.92273452]
 [ 2.99501117]
 [18.1079732 ]]
----------------------------------------------
img_id is 39
{}
[[-0.89731468 -0.18188071 -0.4021763 ]
 [-0.12025014 -0.77595828  0.61921616]
 [-0.42469551  0.6039935   0.67440757]] [[-1.94187744]
 [ 6.87193009]
 [18.19410708]]
----------------------------------------------
img_id is 40
{"b'4'": [1182.0, 647.75], "b'2'": [763.0, 1021.5]}
[[-0.89745625 -0.18308882 -0.40131132]
 [-0.11847908 -0.77630281  0.61912572]
 [-0.4248941   0.60318524  0.67500561]] [[-4.59482555]
 [ 5.12863315]
 [15.76043043]]
----------------------------------------------
img_id is 41
{"b'6'": [1381.25, 79.25]}
[[-0.57941367 -0.81468027  0.02399715]
 [-0.51883786  0.34597916 -0.78173249]
 [ 0.62855952 -0.46539712 -0.62315203]] [[-7.95101439]
 [ 3.16676263]
 [14.8185915 ]]
----------------------------------------------
img_id is 42
{}
[[-0.57723881 -0.81630924  0.02084665]
 [-0.51860434  0.34676327 -0.78154   ]
 [ 0.63074947 -0.46194639 -0.62350673]] [[-5.62321102]
 [ 5.13851365]
 [12.23140589]]
----------------------------------------------
img_id is 43
{"b'2'": [774.75, 1026.5], "b'4'": [1195.5, 652.25]}
[[-0.91106966  0.41221718 -0.00539206]
 [-0.25681963 -0.55728626  0.78960477]
 [ 0.32248373  0.72076974  0.61359207]] [[-8.75261609]
 [ 5.64224716]
 [14.6712986 ]]
----------------------------------------------
img_id is 44
{"b'6'": [1400.25, 80.75]}
[[-0.91164977  0.41092842 -0.00570366]
 [-0.25619755 -0.55741623  0.78971511]
 [ 0.32133707  0.72140486  0.61344723]] [[-7.9734411 ]
 [ 9.15408279]
 [16.42055468]]
----------------------------------------------
img_id is 45
{}
[[-0.9112718   0.41169933 -0.00934712]
 [-0.25914547 -0.55566734  0.78998572]
 [ 0.3200427   0.72231397  0.61305399]] [[-11.48894489]
 [ 11.03642951]
 [ 16.73361049]]
----------------------------------------------
img_id is 46
{}
[[-0.91250221  0.40896483 -0.00935366]
 [-0.25615728 -0.55342415  0.79253085]
 [ 0.3189407   0.72558216  0.60976008]] [[-11.89617557]
 [  7.89329087]
 [ 19.17386512]]
----------------------------------------------
img_id is 47
{"b'2'": [791.75, 1035.25], "b'4'": [1216.5, 659.25], "b'6'": [1424.0, 83.0]}
[[ 0.717423   -0.67526349 -0.17124093]
 [ 0.10375132  0.34663776 -0.93224349]
 [ 0.68886857  0.65104645  0.31874538]] [[-11.48173347]
 [ 11.48960742]
 [ 20.87526014]]
----------------------------------------------
img_id is 48
{"b'4'": [1219.25, 661.0], "b'2'": [794.25, 1038.75], "b'6'": [1427.0, 84.25]}
[[-0.86892746 -0.48683099 -0.08922252]
 [-0.06646037 -0.06386907  0.99574282]
 [-0.49045702  0.87115804  0.02314262]] [[-14.13440348]
 [  8.50802857]
 [ 20.60409712]]
----------------------------------------------
img_id is 49
{"b'4'": [1223.0, 662.5], "b'2'": [797.25, 1039.75], "b'6'": [1432.0, 84.75]}
[[-0.86754051 -0.48964116 -0.08732123]
 [-0.0645553  -0.06322929  0.99590897]
 [-0.49315929  0.86962842  0.02324504]] [[-13.04250878]
 [  4.80939932]
 [ 21.66612707]]
----------------------------------------------
img_id is 50
{"b'4'": [1226.0, 661.75], "b'2'": [799.5, 1040.0], "b'6'": [1438.25, 84.25]}
[[ 0.4843477  -0.87060399  0.08634811]
 [ 0.02232617 -0.0863656  -0.99601332]
 [ 0.87459068  0.48434458 -0.02239373]] [[-15.39967006]
 [  1.80535604]
 [ 20.47469317]]
----------------------------------------------
img_id is 51
{"b'2'": [803.25, 1041.5], "b'6'": [1443.0, 82.75]}
[[ 0.48630599 -0.86952067  0.08625711]
 [ 0.02278125 -0.08606572 -0.99602897]
 [ 0.87349156  0.4863399  -0.02204551]] [[-14.18825823]
 [  4.5961311 ]
 [ 23.07162521]]
----------------------------------------------
img_id is 52
{}
[[ 0.48599215 -0.86973937  0.08581992]
 [ 0.02366714 -0.08506308 -0.99609444]
 [ 0.87364266  0.48612519 -0.02075574]] [[-13.80264723]
 [  6.50958463]
 [ 26.56304322]]
----------------------------------------------
img_id is 53
{}
[[ 0.48557867 -0.87011273  0.08436342]
 [ 0.022753   -0.08389228 -0.99621503]
 [ 0.87389682  0.48566029 -0.02093863]] [[-12.5148419 ]
 [  8.78618945]
 [ 29.58936573]]
----------------------------------------------
img_id is 54
{}
[[ 0.48216667 -0.87218487  0.08251576]
 [ 0.02147338 -0.08239349 -0.99636851]
 [ 0.8758163   0.48218758 -0.02099863]] [[-12.43358468]
 [  5.20930467]
 [ 27.80070911]]
----------------------------------------------
img_id is 55
{}
[[ 0.47711109 -0.87487181  0.08345254]
 [ 0.02065361 -0.08376918 -0.99627112]
 [ 0.87860027  0.47705559 -0.02189795]] [[-11.12699679]
 [  2.40700461]
 [ 30.33841328]]
----------------------------------------------
img_id is 56
{}
[[ 0.47242784 -0.87740323  0.08351957]
 [ 0.01948318 -0.08434169 -0.9962464 ]
 [ 0.88115398  0.47228176 -0.02275076]] [[-10.77710148]
 [ -1.42907816]
 [ 29.26043263]]
----------------------------------------------
img_id is 57
{"b'4'": [1271.5, 674.0]}
[[-0.91586231 -0.26875726 -0.29827129]
 [-0.19139664 -0.36079818  0.91279351]
 [-0.35293562  0.8930813   0.27900222]] [[-10.85914428]
 [ -4.35617467]
 [ 26.5354785 ]]
----------------------------------------------
img_id is 58
{"b'4'": [1273.5, 675.5], "b'6'": [1503.0, 87.25]}
[[-0.91640387 -0.26842585 -0.29690321]
 [-0.19057995 -0.35967329  0.91340813]
 [-0.35197051  0.89363454  0.27844941]] [[-9.38793055]
 [-0.64483375]
 [26.28752952]]
----------------------------------------------